In [1]:
import gspread
import os
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import numpy as np
from datetime import datetime
import openpyxl
from openpyxl import load_workbook

import csv
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

In [2]:
df = pd.read_csv('data.csv')

In [3]:
df.columns

Index(['ShipmentId', 'TierType', 'Type', 'ReverseShipmentId',
       'ForwardReverseType', 'TotalPrice', 'DeliveryPincode', 'Status',
       'DeliveryHub', 'CurrentHub', 'CustomerPromiseDate',
       'LogisticsPromiseDate', 'LatestUpdateTime', 'FirstReceiveTime',
       'FirstReceiveHub', 'LastReceiveTime', 'FirstAssignedHub',
       'NumberOfAttempts', 'CsNotes', 'HubNotes', 'OnHoldByOpsReason',
       'OnHoldByOpsDate', 'BagId', 'BagStatus', 'ConsignmentId', 'OrderId',
       'ItemDescription'],
      dtype='object')

In [4]:
status = pd.pivot_table(df, index='Status',values='ShipmentId', aggfunc='count')

In [5]:
status

,ShipmentId
Status,
Delivered,298
Error,3
Expected,380
Out_For_Delivery,4
Received_By_Ekl,3
Undelivered_HeavyLoad,1
Undelivered_Incomplete_Address,1
Undelivered_No_Response,2
Undelivered_NonServiceablePincode,3


In [6]:
status['per_ShipmentId'] = ((status.div(status['ShipmentId'].sum(),axis=1))*100).round(2)

In [7]:
status

,ShipmentId,per_ShipmentId
Status,,
Delivered,298,39.79
Error,3,0.40
Expected,380,50.73
Out_For_Delivery,4,0.53
Received_By_Ekl,3,0.40
Undelivered_HeavyLoad,1,0.13
Undelivered_Incomplete_Address,1,0.13
Undelivered_No_Response,2,0.27
Undelivered_NonServiceablePincode,3,0.40


In [8]:
# status_table = pd.pivot_table(df, index='Status',values='ShipmentId', aggfunc='count',margins=True)

In [9]:
# status_table

In [10]:
type = pd.pivot_table(df, index=['Status','Type'],values='ShipmentId', aggfunc='count')

In [11]:
type

ShipmentId
Status                                 Type            
Delivered                              COD          153
                                       PP           145
Error                                  COD            2
                                       PP             1
Expected                               COD          232
                                       PP           148
Out_For_Delivery                       COD            3
                                       PP             1
Received_By_Ekl                        COD            2
                                       PP             1
Undelivered_HeavyLoad                  COD            1
Undelivered_Incomplete_Address         COD            1
Undelivered_No_Response                COD            1
                                       PP             1
Undelivered_NonServiceablePincode      COD            2
                                       PP             1
Undelivered_Not_Attended               COD           27
                                       PP            17
Undelivered_Order_Rejected_By_Customer COD            1
                                       PP             1
Undelivered_Request_For_Reschedule     PP             3
Undelivered_SameStateMisroute          COD            1
Untraceable                            COD            2
                                       PP             2

In [12]:
# current_hub

## save pivot in .xlsx

In [13]:
SHEET_NAME = "Summary"

In [14]:
# Load the workbook
wb = openpyxl.Workbook()
wb.save('pivot.xlsx')
wb = openpyxl.load_workbook('pivot.xlsx')

# Check if 'Sheet1' exists
if SHEET_NAME in wb.sheetnames:
    # Delete 'Sheet1'
    del wb[SHEET_NAME]

# Create a new 'Sheet1'
wb.create_sheet(SHEET_NAME)

<Worksheet "Summary">

In [15]:

#save pivots in same sheet

with pd.ExcelWriter('pivot.xlsx', engine='openpyxl') as writer:
    status.to_excel(writer, sheet_name='Summary', startrow=0, startcol=0)  # Pivot 1 in top-left corner
    type.to_excel(writer, sheet_name='Summary', startrow=0, startcol=5)  # Pivot 2 below Pivot 1

print("Pivot tables saved successfully.")

Pivot tables saved successfully.


In [21]:
# pip install openpyxl
# # 

In [24]:
# # Load the existing workbook or create one if it doesn't exist
# wb = openpyxl.load_workbook('pivot.xlsx')

# # Check if 'raw data' sheet exists; if so, remove itA
# if 'raw data' in wb.sheetnames:
#     del wb['raw data']

# # Use ExcelWriter with openpyxl in append mode
# with pd.ExcelWriter('pivot.xlsx', engine='openpyxl', mode='a') as writer:
#     # Assign the workbook to the writer
#     writer.book = wb
#     writer.sheets = {ws.title: ws for ws in wb.worksheets}

#     # Write new data to the 'raw data' sheet
#     df.to_excel(writer, sheet_name='raw data', index=False)

# # Save the updated workbook
# wb.save('pivot.xlsx')

# print("Data successfully updated in 'raw data' sheet.")

## G sheet

In [ ]:
# ## Google Sheets API Authentication
def authenticate():
    creds = None
    # scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
    scope = ["https://www.googleapis.com/auth/spreadsheets", 
             "https://www.googleapis.com/auth/drive"]
    # Check if token.json file exists for previously saved credentials
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", scope)
    
    # If no valid credentials, initiate login process
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file("credentials.json", scope)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open("token.json", "w") as token:
            token.write(creds.to_json())
    client = gspread.authorize(creds)
    
    return client

In [ ]:
SPREADSHEET_ID = "1ejhqkvQNklbPjh6XMGtboiJ7MCyiBSgab5SHHJm0Y7o"
SHEET_NAME = "Summary" 

In [ ]:
df = pd.read_excel('pivot.xlsx', sheet_name=SHEET_NAME)
df.replace([np.nan], '', inplace=True)
# Extract the data ranges
table1 = df.iloc[:, 0:3]  # Range A0:C
table2 = df.iloc[:, 5:8]  # Range F0:H
table3 = df.iloc[:, 10:13]  # Range K0:M

# Combine the extracted tables into a list of lists
table1_data = table1.values.tolist()
table2_data = table2.values.tolist()
table3_data = table3.values.tolist()

In [ ]:
table1

In [ ]:
client = authenticate()

spreadsheet = client.open_by_key(SPREADSHEET_ID)
worksheet = spreadsheet.worksheet('Summary')  # Ensure this sheet exists

# Clear the existing ranges in Google Sheets
worksheet.batch_clear(['A2:C', 'F2:H', 'K2:M'])  # Clear these ranges

# Add new data to the respective ranges
worksheet.update('A2', table1_data)  # Insert table 1 in range A0:C
worksheet.update('F2', table2_data)  # Insert table 2 in range F0:H
worksheet.update('K2', table3_data)  # Insert table 3 in range K0:M

print("Data successfully updated to Google Sheets.")